In [5]:
:dep num = { version = "^0.4.3" }

use num::BigRational as R;
use num::BigInt as I;
use num::BigUint as U;
use num::Integer;
use num::traits::ConstZero;
use num::FromPrimitive;
use num::ToPrimitive;

fn u(i: usize) -> U {
    U::from_usize(i).unwrap()
}

fn rr(i: f64) -> R {
    R::from_float(i).unwrap()
}


fn factorial(n: &U) -> R {
    let mut c = n.clone();
    let one = I::from_i8(1).unwrap();
    let mut out = R::new(one.clone(), one.clone());
    while c > U::ZERO {
        out *= R::new(I::from_biguint(num::bigint::Sign::Plus, c.clone()), one.clone());
        c -= 1u32;
    }
    out
}


In [6]:
/// Считает стационарное распределение вероятностей для пространства
fn stationary_prob_distribution(rho1: &R, rho2: &R, max_c: u8) -> Vec<R> {
    let prob = |x: i32| (rho1 + rho2).pow(x) / (factorial(&u(x as usize)));

    let mut v = Vec::with_capacity(max_c as usize);

    let S = (0..=max_c).map(|x| prob(x as i32)).sum::<R>();
    let Sinv = rr(1.0)/S;

    for c in 0..=max_c {
        let res = &Sinv * prob(c as i32);
        v.push(res);
    }
    v
}

In [20]:
:dep plotters = { version = "^0.3.6", default-features = false, features = ["evcxr", "all_series", "all_elements"] }
extern crate plotters;
use plotters::prelude::*;

let lambda1: R = rr(30.0);
let lambda2: R = rr(10.0);
let mu: R = rr(0.5);
let mu1: R = mu.clone();
let mu2: R = mu.clone();
let rho1: R = &lambda1 / &mu1;
let rho2: R = &lambda2 / &mu2;
let c: u8 = 20;

let dist: Vec<R> = stationary_prob_distribution(&rho1, &rho2, c);
println!("probability distribution: {dist:?}");
println!("Sum should be 1: {}", dist.iter().sum::<R>());

let avg: R = dist.iter().enumerate().map(|(i,v)| rr(i as f64) * v).sum();
println!("Average requests in flight: {avg}");

let time_blocking_prob: R = dist.last().cloned().unwrap();
println!("time blocking probability (E): {}", time_blocking_prob.to_f64().unwrap());

let req_blocking_prob1: R = &lambda1 / (&lambda1 + &lambda2) * &time_blocking_prob;
let req_blocking_prob2: R = &lambda2 / (&lambda1 + &lambda2) * &time_blocking_prob;
println!("request blocking probability: {}, {}", req_blocking_prob1.to_f64().unwrap(), req_blocking_prob2.to_f64().unwrap());



fn draw_chart(data: &Vec<(f32, f32)>, name: impl ToString) -> plotters::evcxr::SVGWrapper {
    let minx = data.iter().min_by(|a, b| a.0.partial_cmp(&b.0).unwrap_or(std::cmp::Ordering::Equal)).unwrap().0;
    let maxx = data.iter().max_by(|a, b| a.0.partial_cmp(&b.0).unwrap_or(std::cmp::Ordering::Equal)).unwrap().0;
    let miny = data.iter().min_by(|a, b| a.1.partial_cmp(&b.1).unwrap_or(std::cmp::Ordering::Equal)).unwrap().1;
    let maxy = data.iter().max_by(|a, b| a.1.partial_cmp(&b.1).unwrap_or(std::cmp::Ordering::Equal)).unwrap().1;
    let figure = evcxr_figure((640, 480), |root| {
        root.fill(&WHITE)?;
        let mut chart = ChartBuilder::on(&root)
            .caption(name.to_string(), ("Arial", 50).into_font())
            .margin(5)
            .x_label_area_size(30)
            .y_label_area_size(30)
            .build_cartesian_2d(minx..maxx, miny..maxy)?;

        chart.configure_mesh().draw()?;

        chart.draw_series(LineSeries::new(
            data.clone(),
            &RED,
        )).unwrap();

        // chart.configure_series_labels()
        //     .background_style(&WHITE.mix(0.8))
        //     .border_style(&BLACK)
        //     .draw()?;
        Ok(())
    });
    return figure;
}

draw_chart(&dist.iter().enumerate().map(|(a,b)| (a as f32, b.to_f32().unwrap())).collect(), "n vs p_n")

probability distribution: [Ratio { numer: 14849255421, denom: 933279301282433386345338108301 }, Ratio { numer: 1187940433680, denom: 933279301282433386345338108301 }, Ratio { numer: 47517617347200, denom: 933279301282433386345338108301 }, Ratio { numer: 1267136462592000, denom: 933279301282433386345338108301 }, Ratio { numer: 25342729251840000, denom: 933279301282433386345338108301 }, Ratio { numer: 405483668029440000, denom: 933279301282433386345338108301 }, Ratio { numer: 5406448907059200000, denom: 933279301282433386345338108301 }, Ratio { numer: 61787987509248000000, denom: 933279301282433386345338108301 }, Ratio { numer: 617879875092480000000, denom: 933279301282433386345338108301 }, Ratio { numer: 5492265556377600000000, denom: 933279301282433386345338108301 }, Ratio { numer: 43938124451020800000000, denom: 933279301282433386345338108301 }, Ratio { numer: 319549996007424000000000, denom: 933279301282433386345338108301 }, Ratio { numer: 2130333306716160000000000, denom: 9332793012

n vs p_n
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
 
0.0
 
 
 
2.0
 
 
 
4.0
 
 
 
6.0
 
 
 
8.0
 
 
 
10.0
 
 
 
12.0
 
 
 
14.0
 
 
 
16.0
 
 
 
18.0
 
 
 
20.0

In [8]:
let mut block_prob = vec![];
let rho2 = R::from_f64(0.0).unwrap();
for lambda1 in 0..=100 {
    let rho1 = rr(lambda1 as f64) / &mu1;
    let dist = stationary_prob_distribution(&rho1, &rho2, c);
    block_prob.push((lambda1 as f32, dist.last().cloned().unwrap().to_f32().unwrap()));
}
block_prob

[(0.0, 0.0), (1.0, 5.8329243e-14), (2.0, 8.277464e-9), (3.0, 3.7250675e-6), (4.0, 0.00015898644), (5.0, 0.0018690499), (6.0, 0.00979564), (7.0, 0.030035483), (8.0, 0.064410925), (9.0, 0.109212846), (10.0, 0.15889196), (11.0, 0.20904599), (12.0, 0.25708255), (13.0, 0.3017891), (14.0, 0.34277654), (15.0, 0.38008487), (16.0, 0.41395226), (17.0, 0.44469154), (18.0, 0.47262853), (19.0, 0.498073), (20.0, 0.521307), (21.0, 0.54258144), (22.0, 0.5621168), (23.0, 0.5801058), (24.0, 0.59671634), (25.0, 0.61209464), (26.0, 0.6263683), (27.0, 0.6396486), (28.0, 0.65203315), (29.0, 0.66360754), (30.0, 0.67444724), (31.0, 0.6846187), (32.0, 0.69418097), (33.0, 0.70318633), (34.0, 0.7116815), (35.0, 0.7197081), (36.0, 0.72730345), (37.0, 0.734501), (38.0, 0.741331), (39.0, 0.7478206), (40.0, 0.75399446), (41.0, 0.75987494), (42.0, 0.7654823), (43.0, 0.77083504), (44.0, 0.77594995), (45.0, 0.78084254), (46.0, 0.7855269), (47.0, 0.79001594), (48.0, 0.7943216), (49.0, 0.7984548), (50.0, 0.80242574), (51

In [19]:
draw_chart(&block_prob, "lambda1 vs E")

lambda1 vs E
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
 
0.0
 
 
 
10.0
 
 
 
20.0
 
 
 
30.0
 
 
 
40.0
 
 
 
50.0
 
 
 
60.0
 
 
 
70.0
 
 
 
80.0
 
 
 
90.0
 
 
 
100.0
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="35,444 40,444 46,444 52,444 58,444 64,444 70,440 76,432 82,417 88,397 94,376 100,354 106,334 112,314 118,297 124,281 130,266 136,253 142,241 148,230 154,220 160,211 166,202 172,195 178,187 184,181 190,175 196,169 202,164 208,159 214,154 220,150 226,145 232,142 238,138 244,134 250,131 256,128 262,125 268,122 274,120 280,117 286,115 292,112 298,110 304,108 310,106 316,104 322,102 328,100 334,99 340,97 346,96 352,94 358,93 364,91 370,90 376,88 382,87 388,86 394,85 400,84 406,82 412,81 418,80 424,79 430,78 436,77 442,76 448,76 454,75 460,74 466,73 472,72 478,71 484,71 490,70 496,69 502,68 508,68 514,67 520,66 526,66 532,65 538,65 544,64 550,63 556,63 562,62 568,62 574,61 580,61 586,60 592,60 598,59 604,59 610,58 616,58 622,57 628,57 634,56 "/>

In [12]:
let lambda2 = rr(4.0);
let c = 121;

let mut avg_req_counts = vec![];
let rho2 = R::from_f64(0.0).unwrap();
for lambda1 in 0..=100 {
    let rho1 = rr(lambda1 as f64) / &mu1;
    let dist = stationary_prob_distribution(&rho1, &rho2, c);
    let avg: R = dist.iter().enumerate().map(|(i,v)| rr(i as f64) * v).sum();
    avg_req_counts.push((lambda1 as f32, avg.to_f32().unwrap()));
}
avg_req_counts

[(0.0, 0.0), (1.0, 2.0), (2.0, 4.0), (3.0, 6.0), (4.0, 8.0), (5.0, 10.0), (6.0, 12.0), (7.0, 14.0), (8.0, 16.0), (9.0, 18.0), (10.0, 20.0), (11.0, 22.0), (12.0, 24.0), (13.0, 26.0), (14.0, 28.0), (15.0, 30.0), (16.0, 32.0), (17.0, 34.0), (18.0, 36.0), (19.0, 38.0), (20.0, 40.0), (21.0, 42.0), (22.0, 44.0), (23.0, 46.0), (24.0, 48.0), (25.0, 50.0), (26.0, 52.0), (27.0, 54.0), (28.0, 56.0), (29.0, 58.0), (30.0, 60.0), (31.0, 62.0), (32.0, 64.0), (33.0, 66.0), (34.0, 68.0), (35.0, 70.0), (36.0, 72.0), (37.0, 73.99999), (38.0, 75.99996), (39.0, 77.999886), (40.0, 79.999664), (41.0, 81.99908), (42.0, 83.997635), (43.0, 85.99436), (44.0, 87.98738), (45.0, 89.9735), (46.0, 91.94757), (47.0, 93.90202), (48.0, 95.82652), (49.0, 97.70814), (50.0, 99.53195), (51.0, 101.282265), (52.0, 102.94409), (53.0, 104.50459), (54.0, 105.9543), (55.0, 107.28772), (56.0, 108.50341), (57.0, 109.60355), (58.0, 110.5932), (59.0, 111.47948), (60.0, 112.27071), (61.0, 112.97579), (62.0, 113.60359), (63.0, 114.1626

In [18]:
draw_chart(&avg_req_counts, "lambda1 vs Nbar")

lambda1 vs Nbar
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0.0
 
 
 
20.0
 
 
 
40.0
 
 
 
60.0
 
 
 
80.0
 
 
 
100.0
 
 
 
 
0.0
 
 
 
10.0
 
 
 
20.0
 
 
 
30.0
 
 
 
40.0
 
 
 
50.0
 
 
 
60.0
 
 
 
70.0
 
 
 
80.0
 
 
 
90.0
 
 
 
100.0
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="35,444 40,438 46,432 52,425 58,419 64,412 70,406 76,399 82,393 88,386 94,380 100,373 106,367 112,360 118,354 124,347 130,341 136,334 142,328 148,321 154,315 160,308 166,302 172,295 178,289 184,282 190,276 196,269 202,263 208,256 214,250 220,243 226,237 232,230 238,224 244,217 250,211 256,204 262,198 268,191 274,185 280,178 286,172 292,165 298,159 304,153 310,146 316,140 322,134 328,127 334,121 340,116 346,110 352,105 358,101 364,96 370,92 376,89 382,86 388,83 394,80 400,78 406,76 412,74 418,72 424,71 430,70 436,68 442,67 448,67 454,66 460,65 466,64 472,64 478,63 484,62 490,62 496,62 502,61 508,61 514,60 520,60 526,60 532,59 538,59 544,59 550,59 556,58 562,58 568,58 574,58 580,58 586,58 592,57 598,57 604,57 610,57 616,57 622,57 628,57 634,56 "/>